# 1. Basic data storage

All programs process data in one form or another, and many need to be able to save and retrieve that data from one invocation to the next. In `Python`, we have many ways to store data. 
- Flat files (e.g. csv, json, xml, etc.)
- RDBMS (e.g. postgres, mysql, etc)
- NoSQL DB (e.g. mongoDB, Neo4J, etc.)

In this tutorial, we will use first flat files, then RDBMS (SQLite), to store and retrieve data. At last we will use `SQLAlchemy (ORM)` which allow you to map rows in a table with python object. `SQLite` is a light weight RDBMS which stores data in a single file without the need for complex installation.

## 1.1 Working with flat files 

### Advantages of Flat Files
- Easy to use and implement
- human-readable makes it easy to edit, view/examine, and transfer 
- support by many language and framework (import, export)


### Disadvantages of Flat Files
- performance: Large files are difficult to view/examine/edit
- No explicit relation between data parts (tables). Relation need to be written in the business logic
- No schema or data structure. Users need understand not only the structure of the data but also the programming tools necessary for accessing it.


In below example, we use pandas to read the csv file. And `treelib` to print the hierarchies

In [8]:
import pandas as pd
from treelib import Node, Tree

In [9]:
def get_data(file_path):
    return pd.read_csv(file_path)

def get_books_by_publisher(data,ascending=True):
    """Return the number of books by each publisher as a pandas series"""
    return data.groupby("publisher").size().sort_values(ascending=ascending)

def get_authors_by_publisher(data, ascending=True):
    """Returns the number of authors by each publisher as a pandas series"""
    return (
        data.assign(name=data.first_name.str.cat(data.last_name, sep=" "))
        .groupby("publisher")
        .nunique()
        .loc[:, "name"]
        .sort_values(ascending=ascending)
    )

def add_new_book(data, author_name, book_title, publisher_name):
    """Adds a new book to the system"""
    # Does the book exist?
    first_name, _, last_name = author_name.partition(" ")
    if any(
        (data.first_name == first_name)
        & (data.last_name == last_name)
        & (data.title == book_title)
        & (data.publisher == publisher_name)
    ):
        return data
    # Add the new book
    return data.append(
        {
            "first_name": first_name,
            "last_name": last_name,
            "title": book_title,
            "publisher": publisher_name,
        },
        ignore_index=True,
    )

def output_author_hierarchy(data):
    """Output the data as a hierarchy list of authors"""
    authors = data.assign(
        name=data.first_name.str.cat(data.last_name, sep=" ")
    )
    authors_tree = Tree()
    authors_tree.create_node("Authors", "authors")
    for author, books in authors.groupby("name"):
        authors_tree.create_node(author, author, parent="authors")
        for book, publishers in books.groupby("title")["publisher"]:
            book_id = f"{author}:{book}"
            authors_tree.create_node(book, book_id, parent=author)
            for publisher in publishers:
                authors_tree.create_node(publisher, parent=book_id)

    # Output the hierarchical authors data
    authors_tree.show()

Below is the main function that calls the above functions 

In [4]:
root_path="../../../data"
file_path=f"{root_path}/author_book_publisher.csv"

data = get_data(file_path)
data.head()

,first_name,last_name,title,publisher
0,Isaac,Asimov,Foundation,Random House
1,Pearl,Buck,The Good Earth,Random House
2,Pearl,Buck,The Good Earth,Simon & Schuster
3,Tom,Clancy,The Hunt For Red October,Berkley
4,Tom,Clancy,Patriot Games,Simon & Schuster


In [5]:
# Get the number of books printed by each publisher
books_by_publisher = get_books_by_publisher(data, ascending=False)
for publisher, total_books in books_by_publisher.items():
    print(f"Publisher: {publisher}, total books: {total_books}")
print()

Publisher: Random House, total books: 4
Publisher: Simon & Schuster, total books: 4
Publisher: Berkley, total books: 2
Publisher: Penguin Random House, total books: 2



In [6]:
# Get the number of authors each publisher publishes
authors_by_publisher = get_authors_by_publisher(data, ascending=False)
for publisher, total_authors in authors_by_publisher.items():
    print(f"Publisher: {publisher}, total authors: {total_authors}")
print()

Publisher: Simon & Schuster, total authors: 4
Publisher: Random House, total authors: 3
Publisher: Berkley, total authors: 2
Publisher: Penguin Random House, total authors: 1



In [10]:
# Output hierarchical authors data
output_author_hierarchy(data)

Authors
├── Alex Michaelides
│   └── The Silent Patient
│       └── Simon & Schuster
├── Carol Shaben
│   └── Into The Abyss
│       └── Simon & Schuster
├── Isaac Asimov
│   └── Foundation
│       └── Random House
├── John Le Carre
│   └── Tinker, Tailor, Soldier, Spy: A George Smiley Novel
│       └── Berkley
├── Pearl Buck
│   └── The Good Earth
│       ├── Random House
│       └── Simon & Schuster
├── Stephen King
│   ├── Dead Zone
│   │   └── Random House
│   ├── It
│   │   ├── Penguin Random House
│   │   └── Random House
│   └── The Shining
│       └── Penguin Random House
└── Tom Clancy
    ├── Patriot Games
    │   └── Simon & Schuster
    └── The Hunt For Red October
        └── Berkley



In [11]:
# Add a new book to the data structure
data = add_new_book(
    data,
    author_name="Pengfei Liu",
    book_title="The real world",
    publisher_name="Random House",
)

/tmp/ipykernel_45614/3122650083.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return data.append(


In [12]:
# Output the updated hierarchical authors data
output_author_hierarchy(data)

Authors
├── Alex Michaelides
│   └── The Silent Patient
│       └── Simon & Schuster
├── Carol Shaben
│   └── Into The Abyss
│       └── Simon & Schuster
├── Isaac Asimov
│   └── Foundation
│       └── Random House
├── John Le Carre
│   └── Tinker, Tailor, Soldier, Spy: A George Smiley Novel
│       └── Berkley
├── Pearl Buck
│   └── The Good Earth
│       ├── Random House
│       └── Simon & Schuster
├── Pengfei Liu
│   └── The real world
│       └── Random House
├── Stephen King
│   ├── Dead Zone
│   │   └── Random House
│   ├── It
│   │   ├── Penguin Random House
│   │   └── Random House
│   └── The Shining
│       └── Penguin Random House
└── Tom Clancy
    ├── Patriot Games
    │   └── Simon & Schuster
    └── The Hunt For Red October
        └── Berkley




## 1.2 Store data in a RDBMS

As we mentioned before, data in flat file has no structure. If we put the `author_book_publisher.csv` data into one table. It can work, but we lose all the advantage of a RDBMS. So first step, we need to do 
data normalization. It often takes three steps:
- convert data to 1st normal form
- convert the result of step1 to 2nd normal form
- convert the result of setp2 to 3rd normal form

Here we will not show the details of how to normalize the tables. 

In below section, we will:
1. create a data model for the csv file
2. create a data base structure (tables with relation) in a RDBMS (SQLite)
3. Populate the DB

The `SQLite database` offers a full-featured relational database management system (RDBMS) that works with a single file to maintain all the database functionality.

### 1.2.1 Create data model

Entities:
- book
- author
- publisher

Relations between entities:

Below figure is an **entity-relation diagram (ERD)**  which is created by using `data grip`.

